In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv', index_col='row_id', parse_dates=['time'])
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv', index_col='row_id', parse_dates=['time'])

In [ ]:
train.head()

## Feature engineering

In [ ]:
le = LabelEncoder()
train['weekday'] = train['time'].dt.weekday
train['time'] = train['time'].dt.time
train['time'] = le.fit_transform(train['time'])
train.head()

In [ ]:
test['weekday'] = test['time'].dt.weekday
test['time'] = test['time'].dt.time
test['time'] = le.transform(test['time'])
test['row_id'] = test.index
test = test.set_index(['time', 'x', 'y', 'direction'])
test.head()

## Predictions

In [ ]:
np.unique(test.weekday)

In [ ]:
test.drop('weekday', axis=1, inplace=True)

Test day is Monday.<br>Let's collect the mean congestion for each (time_interval, x, y, direction) combination across all Mondays in the training set.

In [ ]:
means = pd.DataFrame(train[train['weekday']==0].groupby(['time', 'x', 'y', 'direction']).mean()['congestion'])
means.head()

In [ ]:
test['congestion'] = means['congestion'].reindex(test.index)
test = test.set_index('row_id')
test.head()